# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nanyang,32.9947,112.5328,22.43,37,99,3.26,CN,1681269501
1,1,severo-kurilsk,50.6789,156.1250,-0.40,71,85,4.82,RU,1681269501
2,2,puerto ayora,-0.7393,-90.3518,26.42,89,61,0.45,EC,1681269501
3,3,isiro,2.7739,27.6160,18.73,99,100,1.32,CD,1681269502
4,4,waipawa,-41.4122,175.5153,17.92,59,66,3.28,NZ,1681269503


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "CartoDark",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .8,
    color = "City",
    title =  "Humidities of Various Cities in the World"
   )
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values
def vacrange(values,min,max):
    a=[]
    for value in values:
        if min<value<max:
            a.append(True)
        else:
            a.append(False)
    return a

def alltrue(a,b,c,d):
    e=np.logical_and(a,b)
    f=np.logical_and(e,c)
    g=np.logical_and(f,d)
    return g

ideal_df=city_data_df.loc[alltrue(vacrange(city_data_df['Max Temp'],18,40),vacrange(city_data_df['Humidity'],35,70), vacrange(city_data_df['Cloudiness'],0,50),vacrange(city_data_df['Wind Speed'],0,5)),:]

# Drop any rows with null values
ideal_df.dropna(axis='rows',how='any')

# Display sample data
print(len(ideal_df))
ideal_df.head()

21


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,hithadhoo,-0.6000,73.0833,28.10,69,11,0.58,MV,1681269509
63,63,airai,-8.9266,125.4092,23.83,62,30,0.86,TL,1681269522
92,92,port blair,11.6667,92.7500,30.85,54,45,1.61,IN,1681269530
122,122,avarua,-21.2078,-159.7750,28.03,65,20,2.06,CK,1681269536
129,129,nambucca heads,-30.6500,153.0000,25.65,43,14,2.54,AU,1681269537


### Step 3: Create a new DataFrame called `hotel_df`.

In [91]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_df.iloc[:,[1,8,2,3,5]]
hotel_df.reset_index(inplace=True, drop=True)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

c:\Users\Skynet\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hithadhoo,MV,-0.6000,73.0833,69,
1,airai,TL,-8.9266,125.4092,62,
2,port blair,IN,11.6667,92.7500,54,
3,avarua,CK,-21.2078,-159.7750,65,
4,nambucca heads,AU,-30.6500,153.0000,43,
5,tomohon,ID,1.3346,124.8392,58,
6,japura,ID,-0.3167,102.3500,62,
7,jertih,MY,5.7336,102.4897,55,
8,yago,MX,21.8333,-105.0667,52,
9,lumphat,KH,13.4878,106.9816,44,


In [59]:
hotel_df.iloc[0,3]

73.0833

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [103]:
# Set parameters to search for a hotel
limit = 1
radius = 10000
longitude=0
latitude=0


# set up a parameters dictionary
params = {"apiKey":geoapify_key, 'categories':'accommodation.hotel', 'limit':limit}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.iloc[index,2]
    longitude=hotel_df.iloc[index,3]
    print(latitude,longitude)
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
-0.6 73.0833


c:\Users\Skynet\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hithadhoo - nearest hotel: Pebbles Inn
-8.9266 125.4092
Airai - nearest hotel: No hotel found
11.6667 92.75
Port Blair - nearest hotel: Andaman Ocean Breeze
-21.2078 -159.775
Avarua - nearest hotel: Paradise Inn
-30.65 153.0
Nambucca Heads - nearest hotel: Star Hotel
1.3346 124.8392
Tomohon - nearest hotel: Okoy Flower Garden Resort
-0.3167 102.35
Japura - nearest hotel: No hotel found
5.7336 102.4897
Jertih - nearest hotel: Hotel Rahima
21.8333 -105.0667
Yago - nearest hotel: No hotel found
13.4878 106.9816
Lumphat - nearest hotel: No hotel found
2.3 111.8167
Sibu - nearest hotel: The Paramount Hotel
28.6833 115.8833
Nanchang - nearest hotel: 锦江之星
3.1667 113.0333
Bintulu - nearest hotel: Bakun Inn
-25.2986 152.8535
Hervey Bay - nearest hotel: Hervey Bay Hotel
-41.45 147.1667
Launceston - nearest hotel: Hotel Charles
-28.5708 -70.7581
Vallenar - nearest hotel: Hotel del Marqués
-8.0089 -34.8553
Olinda - nearest hotel: Fidji Motel
-30.0392 -52.8939
Cachoeira Do Sul - nearest hotel: Hote

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Hithadhoo,MV,-0.6000,73.0833,69,Pebbles Inn
1,Airai,TL,-8.9266,125.4092,62,No hotel found
2,Port Blair,IN,11.6667,92.7500,54,Andaman Ocean Breeze
3,Avarua,CK,-21.2078,-159.7750,65,Paradise Inn
4,Nambucca Heads,AU,-30.6500,153.0000,43,Star Hotel
5,Tomohon,ID,1.3346,124.8392,58,Okoy Flower Garden Resort
6,Japura,ID,-0.3167,102.3500,62,No hotel found
7,Jertih,MY,5.7336,102.4897,55,Hotel Rahima
8,Yago,MX,21.8333,-105.0667,52,No hotel found
9,Lumphat,KH,13.4878,106.9816,44,No hotel found


In [104]:
#Title case city names
hotelTC_df=hotel_df
hotelTC_df['City']=hotel_df['City'].str.title()


c:\Users\Skynet\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [105]:
%%capture --no-display

# Configure the map plot

map_plot = hotelTC_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "StamenWatercolor",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    color = "City",
    title =  "Hotels in Cities That Fit My Vacation Weather Preferences",
    hover_cols=['Hotel Name','Country']
   )
# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)